<img src="logo.png" width="420px">

# <span style="font-family:Cascadia mono; color:#db5a5a">TODOIST AUTOMATION - DIEGO</span>

## <span style="font-family:Cascadia mono; color:#db6a6a">Settings</span>

### Libraries

In [1]:
from todoist_api_python.api import TodoistAPI
from datetime import datetime, timedelta, timezone
import functions as fun
import random
import json
import time
import os
from dotenv import load_dotenv

load_dotenv()

True

### Message

In [2]:
today = datetime.today()

In [3]:
messages = [f'Todoist Automation for {today.strftime("%Y-%m-%d")}']

### Runtime

In [4]:
start_time = time.time()

### API

In [5]:
api_token = os.getenv("API_TOKEN")
api = TodoistAPI(api_token)

## <span style="font-family:Cascadia mono; color:#db6a6a">Data</span>

#### Projects

In [6]:
all_projects = fun.getProjects()
projects_dict_id, projects_dict_name = fun.getProjectsDicts(all_projects)

#### Tasks

In [7]:
def refreshTasks():
    all_tasks = fun.getTasks()
    task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)
    return all_tasks, task_dict_id, task_dict_name

In [8]:
all_tasks, task_dict_id, task_dict_name = refreshTasks()

{'full_sync': True, 'items': [{'added_at': '2021-02-15T18:44:07.698173Z', 'added_by_uid': '32572547', 'assigned_by_uid': None, 'checked': False, 'child_order': 5, 'collapsed': False, 'completed_at': None, 'content': 'Examen Claudia', 'day_order': -1, 'description': '', 'due': None, 'duration': None, 'id': '4582634104', 'is_deleted': False, 'labels': [], 'parent_id': None, 'priority': 4, 'project_id': '2273727343', 'responsible_uid': None, 'section_id': None, 'sync_id': None, 'updated_at': '1970-01-01T00:00:00Z', 'user_id': '32572547', 'v2_id': '6J2qhHX95GRV4xrR', 'v2_parent_id': None, 'v2_project_id': '6G8RcV29FxfGFg36', 'v2_section_id': None}, {'added_at': '2021-02-15T18:54:05.130585Z', 'added_by_uid': '32572547', 'assigned_by_uid': None, 'checked': False, 'child_order': 14, 'collapsed': False, 'completed_at': None, 'content': 'Hacer cronos', 'day_order': -1, 'description': '', 'due': None, 'duration': None, 'id': '4582658679', 'is_deleted': False, 'labels': ['Med', 'PC', 'Recurring']

#### Sections

In [9]:
all_sections = fun.getSections()
sections_dict_id, sections_dict_name = fun.getSectionsDicts(all_sections)

#### Test

In [10]:
index = random.randint(0,len(all_tasks))
try:
    print(all_tasks[index]['content'], "-", all_tasks[index]['due']['date'], "-", projects_dict_id[all_tasks[index]['project_id']])
except TypeError:
    print(all_tasks[index]['content'], "-", "No date" , "-",projects_dict_id[all_tasks[index]['project_id']],)

GW33 Bournemouth (A) - 2024-04-14T17:30:00 - Fútbol


#### Labels

In [11]:
label_names=[]
for task in all_tasks:
    gross_labels=task['labels']
    for label in gross_labels:
        if label not in label_names:
            label_names.append(label)
label_names

['Med',
 'PC',
 'Recurring',
 'Phone',
 'Short',
 'Home',
 'Long',
 'Outside',
 '❌_streak',
 '✅_streak',
 'Vacations']

#### F1 Calendar

In [12]:
def getF1Calendar():
    return True


## <span style="font-family:Cascadia mono; color:#db6a6a">Orders</span>

#### <span style="font-family:Cascadia mono; color:silver">Create and edit tasks</span>

In [13]:
def createTask(
    content = "Tarea creada automáticamente",
    description = None,
    project_id = projects_dict_name['Inbox'],
    section_id = None,
    parent_id = None,
    order = 1,
    labels = [],
    priority = 1,
    comment_count = 0,
    due_string = None,
    due_date = None,
    due_datetime = None,
    due_lang = None,
    assignee_id = None,
    duration = None,
    duration_unit = None
):
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        task = api.add_task(
            assignee_id = assignee_id,
            comment_count = comment_count,
            content = content,
            description = description,
            due_string = due_string,
            due_date = due_date,
            due_datetime = due_datetime,
            due_lang = due_lang,
            duration = duration,
            labels = labels,
            order = order,
            priority = fun.priorityInversal(priority),
            project_id = project_id,
            section_id = section_id,
            parent_id = parent_id,
            duration_unit = duration_unit
        )
        message = f'Task "{task.content}" ({task.id}) was created correctly'
        print(message)
        return message
    except Exception as error:
        print (error)
        return error
        
def editTask(
    task_id,
    content = None,
    assignee_id = None,
    description = None,
    due_date = None,
    due_string = None,
    due_datetime = None,
    labels = None,
    priority = None,
):
    task = api.get_task(task_id)
    labels = task.labels if labels == None else labels
    content = task.content if content == None else content
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        task = api.update_task(
            task_id = task_id,
            assignee_id = assignee_id,
            content = content,
            description = description,
            labels = labels,
            priority = priority,
            due_date = due_date,
            due_string = due_string,
            due_datetime = due_datetime,
        )
        message = f'Task {content} updated correctly to {task["content"]}'
        print(message)
        return message
    except Exception as error:
        print(message)
        return error

#### <span style="font-family:Cascadia mono; color:silver">Add duration labels</span>

In [14]:
def updateTaskDurationLabel(task_id, task_duration):
    duration_label = fun.getDurationLabel(task_duration)
    task_id = task_id[0]
    task_labels_without_duration = fun.getLabelsWithoutDuration(task_id)
    task_labels_without_duration.append(duration_label)
    return editTask(task_id=task_id,
             labels=task_labels_without_duration)

In [15]:
to_update = False
for task in all_tasks:
    if task['duration'] != None and task['due'] != None:
        if fun.getDurationLabel(task['duration']['amount']) not in task['labels']:
            to_update = True
            messages.append('Tasks to add duration label...')
            message = updateTaskDurationLabel([task['id']], task['duration']['amount'])
            messages.append(message)

if not to_update:
    message = 'No tasks to add duration label'
    print(message)
    messages.append(message)
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

No tasks to add duration label


#### <span style="font-family:Cascadia mono; color:silver">Capitalize titles</span>

In [16]:
to_update = False
for task in all_tasks:
    if task["content"][0].upper() != task["content"][0]:
        to_update = True
        task_id = task["id"]
        messages.append("Tasks to capitalize it's title...")
        message = editTask(task_id = task_id,
                           content = task["content"][0].upper() + task["content"][1:])
        messages.append(message)

if not to_update:
    message = "No task to capitalize it's title"
    print(message)
    messages.append(message)
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

No task to capitalize it's title


#### <span style="font-family:Cascadia mono; color:silver">Uncomplete recurring tasks</span>

In [17]:
with open('../recurringtasksdiego', 'r') as archivo_json:
    recurringtasks = json.load(archivo_json)

In [18]:
to_update = False
for task_id in recurringtasks.keys():
    if fun.getTask(task_id).is_completed:
        to_update = True
        print(task.id)
        fun.uncompleteTask(task_id)
        messages.append('Tasks to add uncomplete...')
        message = editTask(task_id=task_id,
                 due_string="No date")
        messages.append(message)
        
if not to_update:
    message = "No recurring tasks to uncomplete"
    print(message)
    messages.append(message)
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

No tasks to uncomplete


#### <span style="font-family:Cascadia mono; color:silver">Add labels to birthday tasks</span>

In [19]:
to_update = False
for task in all_tasks:
    if task['project_id']==projects_dict_name['Cumpleaños'] and task['labels']!=['Phone','Short']:
        to_update = True
        messages.append('Birthday tasks to add labels...')
        message = editTask(task_id=task['id'],
                 labels=['Phone','Short'])
        messages.append(message)
        
if not to_update:
    message = "No birthday tasks to add labels"
    print(message)
    messages.append(message)
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

No birthday tasks to add labels


#### <span style="font-family:Cascadia mono; color:silver">Reminder and completion of habit tracking tasks not being completed</span>

In [20]:
# fun.completeTask('')

#### <span style="font-family:Cascadia mono; color:silver">Fantasy task due</span>

In [21]:
evaluate = True
if fun.getTask('4632052423').is_completed == True:
    editTask('4632052423', due_string='every friday 20:00')
    message = 'Fantasy task moved back to fridays'
    print (message)
    messages.append(message)
    evaluate = False

In [22]:
to_update = False
if evaluate:
    if today.weekday() in [0,5,6] and fun.getTask('4632052423').due.date != today.strftime('%Y-%m-%d'):
        for task in all_tasks:
            if task['section_id'] == '51988025' and task['parent_id'] == '6968182312':
                try:
                    fantasydate = datetime.strptime(fun.getTask('4632052423').due.date, '%Y-%m-%d')
                    matchday = datetime.strptime(task['due']['date'][:10], '%Y-%m-%d')
                    if fantasydate > matchday:
                        message = 'Fantasy task moved to Tuesday'
                        messages.append(message)
                        editTask('4632052423', due_string="Tuesday")
                        to_update = True
                        all_tasks, task_dict_id, task_dict_name = refreshTasks()
                        break
                except TypeError:
                    pass
    if not to_update:
        message = 'Fantasy not moved'
        print(message)
        messages.append(message)

Fantasy not moved


## <span style="font-family:Cascadia mono; color:#db6a6a">Results</span>

### Export all tasks and it's status

In [23]:
tasks = api.get_tasks()
recurringtasks={}
for task in tasks:
    if 'Recurring' in task.labels:
        recurringtasks[task.id]=task.is_completed
with open('../recurringtasksdiego', 'w') as file:
    json.dump(recurringtasks, file)

### Runtime

In [24]:
end_time = time.time()

In [25]:
runtime = f'Runtime: {round(end_time- start_time,3)} seconds'
messages.append(runtime)
print(runtime)

Runtime: 42.237 seconds


### <span style="font-family:Cascadia code; color:#7a3aca">Message</span>

In [28]:
body = messages[0] + "\n\n"
for message in messages[1:]:
    body = body + "- " + message + "\n"
    print(message,"\n")
print(runtime)

No tasks to add duration label 

No task to capitalize it's title 

No tasks to uncomplete 

No birthday tasks to add labels 

Fantasy not moved 

Runtime: 42.237 seconds 

Runtime: 42.237 seconds


In [29]:
fun.sendEmail('diegorodgar17@gmail.com', 'Daily Todoist', body)

Email sent correctly
